In [ ]:
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
from collections import Counter
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
import plotly.express as px

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sheet_url = 'https://docs.google.com/spreadsheets/d/1AC_OCRf_JBEl5kcPNHK-zU_pYLIqVvdFnka9Qj9G8tM/edit?usp=sharing'
# Extract sheet ID and gid from the URL
sheet_id = sheet_url.split('/d/')[1].split('/edit')[0]
gid = '0'

In [ ]:
try:
    sheet_name = 'roles'
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    df_roles = pd.read_csv(url)
    print("Dataset imported successfully!\n")
    print(df_roles.head())

except FileNotFoundError:
    print("Error: File not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Dataset imported successfully!

                 university       role  \
0  Arizona State University      Staff   
1  Arizona State University      Staff   
2  Arizona State University      Staff   
3  Arizona State University  Librarian   
4  Arizona State University  Librarian   

                                               title notes  
0                           Data Analysis Specialist   NaN  
1        Data Analysis Specialist, Library Analytics   NaN  
2                            Data Science Specialist   NaN  
3                    Digital Humanities Data Analyst   NaN  
4  Director of Maps, Imagery, and Geospatial Serv...   NaN  


In [ ]:
try:
    sheet_name = 'recap'
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'

    df_recap = pd.read_csv(url)
    print("Dataset imported successfully!")
    print(df_recap.head())

except FileNotFoundError:
    print("Error: File not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Dataset imported successfully!
                           university   status  total  dataroles  librarian  \
0            Arizona State University   Public    183          6          3   
1                   Boston University  Private     91          0          0   
2                 Brandeis University  Private     56          2          2   
3                    Brown University  Private    133          2          2   
4  California Institute of Technology  Private     42          1          0   

   staff  other  data  digital  gis  research  \
0      3      0     5        1    0         0   
1      0      0     0        0    0         0   
2      0      0     2        0    0         0   
3      0      0     2        0    1         0   
4      1      0     1        0    0         1   

                                 web  \
0                                  -   
1                                  -   
2                      Data Services   
3  Data & Statistics Research Guides   

# Prevalence of Data-Related Roles

In [ ]:
# Calculate percentages
df_recap['pctdataroles'] = df_recap['dataroles']*100/df_recap['total']
df_recap['pctdatalib'] = df_recap['librarian']*100/df_recap['dataroles']
df_recap['pctdatastaff'] = df_recap['staff']*100/df_recap['dataroles']
df_recap['pctdataother'] = df_recap['other']*100/df_recap['dataroles']

In [ ]:
# Create the table
table = pd.pivot_table(df_recap, values=['total', 'dataroles', 'pctdataroles', 'librarian', 'pctdatalib', 'staff', 'pctdatastaff', 'other', 'pctdataother'], index=['university', 'status'], aggfunc='sum')

# Reorder the column
column_order = ['total', 'dataroles', 'pctdataroles', 'librarian', 'pctdatalib', 'staff', 'pctdatastaff', 'other', 'pctdataother']
table = table.reindex(columns=column_order, level=0)

table.round(2)


,,total,dataroles,pctdataroles,librarian,pctdatalib,staff,pctdatastaff,other,pctdataother
university,status,,,,,,,,,
Arizona State University,Public,183,6,3.28,3,50.00,3,50.00,0,0.0
Boston University,Private,91,0,0.00,0,0.00,0,0.00,0,0.0
Brandeis University,Private,56,2,3.57,2,100.00,0,0.00,0,0.0
Brown University,Private,133,2,1.50,2,100.00,0,0.00,0,0.0
California Institute of Technology,Private,42,1,2.38,0,0.00,1,100.00,0,0.0
...,...,...,...,...,...,...,...,...,...,...
University of Washington,Public,305,9,2.95,9,100.00,0,0.00,0,0.0
University of Wisconsin-Madison,Public,283,3,1.06,3,100.00,0,0.00,0,0.0
Vanderbilt University,Private,152,3,1.97,3,100.00,0,0.00,0,0.0


In [ ]:
total_all = df_recap['total'].sum()
dataroles_all = df_recap['dataroles'].sum()
librarian_all = df_recap['librarian'].sum()
staff_all = df_recap['staff'].sum()
other_all = df_recap['other'].sum()

pctdataroles_all = (dataroles_all / total_all) * 100 if total_all else 0
pctdatalib_all = (librarian_all / dataroles_all) * 100 if dataroles_all else 0
pctdatastaff_all = (staff_all / dataroles_all) * 100 if dataroles_all else 0
pctdataother_all = (other_all / dataroles_all) * 100 if dataroles_all else 0

# Create a dictionary with the values
data = {
    'Total': [total_all],
    'Data Roles': [dataroles_all],
    'Percentage of Data Roles': [pctdataroles_all],
    'Data Librarian': [librarian_all],
    'Percentage of Librarians in Data Roles': [pctdatalib_all],
    'Staff': [staff_all],
    'Percentage of Staff in Data Roles': [pctdatastaff_all],
    'Other': [other_all],
    'Percentage of Other in Data Roles': [pctdataother_all]
}

# Create the DataFrame
df_summary = pd.DataFrame(data)

# Display the table
df_summary.round(2)

,Total,Data Roles,Percentage of Data Roles,Data Librarian,Percentage of Librarians in Data Roles,Staff,Percentage of Staff in Data Roles,Other,Percentage of Other in Data Roles
0,14470,321,2.22,209,65.11,105,32.71,7,2.18


# Public vs. Private Comparisons

In [ ]:
# Calculate summary statistics for Table 2
def summarize_data(df):
    summary = df.agg(['mean', 'std', 'median'])
    return summary

# Overall summary statistics
overall_summary = summarize_data(df_recap[['total', 'dataroles', 'pctdataroles', 'librarian', 'staff', 'other', 'pctdatalib']])
print("Overall Summary Statistics:")
print(overall_summary.round(2))

# Summary statistics by status
public_summary = summarize_data(df_recap[df_recap['status'] == 'Public'][['total', 'dataroles', 'pctdataroles', 'librarian', 'staff', 'other', 'pctdatalib']])
print("\nPublic Summary Statistics:")
print(public_summary.round(2))
private_summary = summarize_data(df_recap[df_recap['status'] == 'Private'][['total', 'dataroles', 'pctdataroles', 'librarian', 'staff', 'other', 'pctdatalib']])
print("\nPrivate Summary Statistics:")
print(private_summary.round(2))


Overall Summary Statistics:
         total  dataroles  pctdataroles  librarian  staff  other  pctdatalib
mean    203.80       4.52          2.31       2.94   1.48   0.10       70.25
std     117.74       3.54          1.40       2.19   1.99   0.45       32.01
median  175.00       4.00          2.08       3.00   1.00   0.00       75.00

Public Summary Statistics:
         total  dataroles  pctdataroles  librarian  staff  other  pctdatalib
mean    210.48       4.20          2.12       2.92   1.20   0.08       72.20
std     112.47       2.95          1.24       2.18   1.51   0.47       31.81
median  186.00       3.50          1.99       2.50   1.00   0.00       81.67

Private Summary Statistics:
         total  dataroles  pctdataroles  librarian  staff  other  pctdatalib
mean    195.19       4.94          2.55       2.97   1.84   0.13       67.47
std     125.56       4.19          1.57       2.24   2.46   0.43       32.67
median  152.00       4.00          2.33       3.00   1.00   0.00    

In [ ]:
def compare_groups(df, column):
    df_filtered = df.dropna(subset=[column])  # Remove rows with missing values in the selected column
    private_group = df_filtered[df_filtered['status'] == 'Private'][column]
    public_group = df_filtered[df_filtered['status'] == 'Public'][column]

    if private_group.empty or public_group.empty:
        print(f"Skipping {column}: One of the groups is empty after dropping NaN values.")
        return

    # Perform Shapiro-Wilk test for normality
    stat_private, p_private = shapiro(private_group)
    stat_public, p_public = shapiro(public_group)

    alpha = 0.05

    # Perform t-test and Mann-Whitney U test
    t_statistic, p_value_t = ttest_ind(private_group, public_group)
    statistic, p_value_u = mannwhitneyu(private_group, public_group)

    # Print results
    print(f"t-test for {column}: t_statistic = {t_statistic:.2f}, p-value = {p_value_t:.3f}")
    print(f"Mann-Whitney U test for {column}: statistic = {statistic:.2f}, p-value = {p_value_u:.3f}")

    if p_private > alpha and p_public > alpha:  # Both groups are normally distributed
        print("Both groups are normally distributed, use t-test")
    else:
        print("Normality assumptions are not met, use Mann-Whitney U test")
    print("")

# Perform tests for each relevant column
for col in ['total', 'dataroles', 'pctdataroles', 'librarian', 'pctdatalib']:
    compare_groups(df_recap, col)

t-test for total: t_statistic = -0.54, p-value = 0.591
Mann-Whitney U test for total: statistic = 535.00, p-value = 0.327
Normality assumptions are not met, use Mann-Whitney U test

t-test for dataroles: t_statistic = 0.87, p-value = 0.389
Mann-Whitney U test for dataroles: statistic = 661.50, p-value = 0.632
Normality assumptions are not met, use Mann-Whitney U test

t-test for pctdataroles: t_statistic = 1.27, p-value = 0.210
Mann-Whitney U test for pctdataroles: statistic = 739.00, p-value = 0.169
Normality assumptions are not met, use Mann-Whitney U test

t-test for librarian: t_statistic = 0.08, p-value = 0.936
Mann-Whitney U test for librarian: statistic = 644.50, p-value = 0.776
Normality assumptions are not met, use Mann-Whitney U test

t-test for pctdatalib: t_statistic = -0.60, p-value = 0.552
Mann-Whitney U test for pctdatalib: statistic = 507.50, p-value = 0.497
Normality assumptions are not met, use Mann-Whitney U test



# Text Analysis of Job Titles

In [ ]:
# Job title text to analyze
text_data = df_roles['title'].dropna().astype(str)
total_rows = len(text_data)

# Tokenize the text and create n-grams
def generate_ngrams(text, n):
    tokens = word_tokenize(text.lower())
    stop_words = set(nltk.corpus.stopwords.words('english'))
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return list(ngrams(tokens, n))

unigrams = []
bigrams = []
trigrams = []
for text in text_data:
    unigrams.extend(generate_ngrams(text, 1))
    bigrams.extend(generate_ngrams(text, 2))
    trigrams.extend(generate_ngrams(text, 3))

# Format ngram labels
def clean_label(ngram):
    if isinstance(ngram, tuple):
        return ' '.join(ngram)  # Join tuple elements with space
    else:
        return ngram

def plot_ngrams(ngrams, title):
    freq_dist = Counter(ngrams)
    top_10 = freq_dist.most_common(10)

    labels = [clean_label(ngram) for ngram, freq in top_10]  # Clean labels
    counts = [freq for _, freq in top_10]  # Raw frequencies for bar visualization
    percentages = [(freq / total_rows) * 100 for _, freq in top_10]  # Calculate % based on # rows
    percentage_labels = [f"{freq} ({p:.2f}%)" for freq, p in zip(counts, percentages)]  # Add both count and % as labels

    fig = px.bar(
        x=counts,
        y=labels,
        orientation='h',
        labels={'x': 'Frequency', 'y': 'N-grams'},
        title=title
    )

    fig.update_traces(text=percentage_labels, textposition='inside')  # Show % labels
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})  # Reverse sorting

    fig.show()

plot_ngrams(unigrams, 'Top 10 Unigrams')
plot_ngrams(bigrams, 'Top 10 Bigrams')
plot_ngrams(trigrams, 'Top 10 Trigrams')


## Generalist vs Specialist

In [ ]:
# Define keyword lists
leadership_keywords = {'director', 'head', 'chief'}
specialist_keywords = {'geospatial', 'geographic', 'gis', 'visualization', 'quantitative', 'qualitative', 'statistic', 'survey', 'text', 'bio', 'specialist'}
generalist_keywords = {'data', 'research', 'digital', 'services', 'librarian'}

# Function to classify job titles
def classify_role(title):
    title_lower = title.lower()

    is_generalist = any(keyword in title_lower for keyword in generalist_keywords)
    is_specialist = any(keyword in title_lower for keyword in specialist_keywords)
    is_leadership = any(keyword in title_lower for keyword in leadership_keywords)

    if is_leadership and is_specialist:
        return 'Leadership, Specialist'
    elif is_leadership:
        return 'Leadership, Generalist'
    elif is_specialist:
        return 'Specialist'
    elif is_generalist:
        return 'Generalist'
    else:
        return 'Other'

# Apply classification to job titles
df_roles['Role Classification'] = df_roles['title'].apply(classify_role)

# Count occurrences of each classification
classification_counts = df_roles['Role Classification'].value_counts()

# Calculate percentages
classification_percentages = (classification_counts / total_rows) * 100

# Prepare data for visualization
classification_df = pd.DataFrame({
    'Role Type': classification_counts.index,
    'Count': classification_counts.values,
    'Percentage': classification_percentages.values
})

# Create bar chart
fig = px.bar(
    classification_df,
    x='Count',
    y='Role Type',
    orientation='h',
    text=classification_df.apply(lambda row: f"{row['Count']} ({row['Percentage']:.2f}%)", axis=1),
    title='Comparison of Generalist vs. Specialist Data Roles',
    labels={'Count': 'Frequency', 'Role Type': 'Classification'}
)

fig.update_traces(textposition='inside')
fig.update_layout(yaxis={'categoryorder': 'total ascending'}, width=1200, height=700)

fig.show()


# Text Analysis of Web Presence

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def analyze_web_presence(df):
    # Initialize counters
    results = {
        'All': {'Exact "Data Services"': 0, 'Includes "Data" and "Services"': 0,
                'Includes "Data"': 0, 'Alternative Phrasing': 0, 'No Page': 0},
        'Public': {'Exact "Data Services"': 0, 'Includes "Data" and "Services"': 0,
                   'Includes "Data"': 0, 'Alternative Phrasing': 0, 'No Page': 0},
        'Private': {'Exact "Data Services"': 0, 'Includes "Data" and "Services"': 0,
                    'Includes "Data"': 0, 'Alternative Phrasing': 0, 'No Page': 0}
    }

    total_rows = len(df)  # Total rows for percentage calculation

    for _, row in df.iterrows():
        web_text = str(row['web']).lower()
        status = row['status']

        # Categorize the web presence
        if web_text == "data services":
            category = 'Exact "Data Services"'
        elif "data" in web_text and "services" in web_text:
            category = 'Includes "Data" and "Services"'
        elif "data" in web_text:
            category = 'Includes "Data"'
        elif "-" == web_text:
            category = 'No Page'
        else:
            category = 'Alternative Phrasing'

        results['All'][category] += 1
        results[status][category] += 1  # Update counts based on Private/Public

    # Convert counts to DataFrame
    summary_df = pd.DataFrame(results)

    # Calculate percentages based on total rows
    percentage_df = summary_df.copy()
    for col in percentage_df.columns:
        percentage_df[col] = (summary_df[col] / total_rows) * 100  # Convert to %

    # Run separate chi-square tests for each classification
    chi_square_results = []
    for category in results['All'].keys():
        observed = pd.DataFrame({
            'Public': [results['Public'][category], len(df[df['status'] == 'Public']) - results['Public'][category]],
            'Private': [results['Private'][category], len(df[df['status'] == 'Private']) - results['Private'][category]]
        }, index=[f"Has {category}", f"Does Not Have {category}"])

        # Perform chi-square test
        chi2_stat, p_value, dof, expected = chi2_contingency(observed)

        # Check assumption: expected frequencies should be ≥ 5
        assumption_check = "Warning: Expected freq < 5" if (expected < 5).any() else "Chi-square assumptions met"

        # Store results
        chi_square_results.append({
            'Category': category,
            'Chi-Square Statistic': round(chi2_stat, 2),
            'p-value': round(p_value, 3),
            'Degrees of Freedom': dof,
            'Assumption Check': assumption_check,
            'Conclusion': "Significant association (reject independence)" if p_value < 0.05 else "No significant association (fail to reject independence)"
        })

    chi_square_df = pd.DataFrame(chi_square_results)

    return summary_df, percentage_df, chi_square_df

# Analyze and display results
counts_df, percentages_df, chi_square_df = analyze_web_presence(df_recap)

# Display results
print("Counts Data:")
print(counts_df, "\n")

print("Percentages Data:")
print(percentages_df.round(2), "\n")

print("Chi-Square Test Results for Each Classification:")
print(chi_square_df)


Counts Data:
                                All  Public  Private
Exact "Data Services"            15       7        8
Includes "Data" and "Services"   15       7        8
Includes "Data"                  16       9        7
Alternative Phrasing              3       1        2
No Page                          22      16        6 

Percentages Data:
                                  All  Public  Private
Exact "Data Services"           21.13    9.86    11.27
Includes "Data" and "Services"  21.13    9.86    11.27
Includes "Data"                 22.54   12.68     9.86
Alternative Phrasing             4.23    1.41     2.82
No Page                         30.99   22.54     8.45 

Chi-Square Test Results for Each Classification:
                         Category  Chi-Square Statistic  p-value  \
0           Exact "Data Services"                  0.31    0.577   
1  Includes "Data" and "Services"                  0.31    0.577   
2                 Includes "Data"                  0.00    1.000